# SnapPy and SageMath are friends!

* SnapPy always uses part of Sage, namely its interface to PARI, repackaged into the stand-alone module CyPari:

    - Smith normal form for homology computations.

    - Arbitrary precision arithmetic.

    - Available on PyPI.
    

* When installed into Sage, via:
```
sage -pip install --no-use-wheel snappy
```
SnapPy gains additional functionality.  GUI still works:
```
sage -python -m snappy.app
```


* Installed on SageMathCloud.
   

## Sage-specific features.

Numeric return types are native Sage types.

In [ ]:
%gui tk
import snappy
M = snappy.Manifold('m004')
type(M.volume())

In [ ]:
MH = M.high_precision()
MH.volume()

In [ ]:
MH.volume().prec() # quad-double

In [ ]:
M.tetrahedra_shapes('rect', bits_prec=1000)[0]

The quad-double precision kernel allows computing very complicated Dirichlet domains.  



In [ ]:
B = snappy.Manifold('13a100')
print(B.volume())
try:
    B.dirichlet_domain()
except RuntimeError, error:
    print(error)

In [ ]:
BH = B.high_precision()
D = BH.dirichlet_domain(); D
#B.plink(), D.view()

Because of local rigidity, the shapes are always algebraic numbers.  We can use LLL to recover their exact expressions, following Goodman, Neumann, et. al.

In [ ]:
TF = M.tetrahedra_field_gens()
TF

In [ ]:
TF.find_field(100, 10, True)

Finite covers are of central interest in 3-manifold theory, e.g. Agol's recent solution to the Virtual Haken Conjecture. SnapPy has some basic ability here:

In [ ]:
%time len(M.covers(9))

However, in Sage one can use GAP or Magma to explore *much* bigger covers.

In [ ]:
from sage.all import gap, magma, PSL
gap(1), magma(1) # one-time startup cost
%time covers = M.covers(9, method='gap')

In [ ]:
%time covers = M.covers(9, method='magma')

In [ ]:
[C.homology() for C in covers]

In [ ]:
G = M.fundamental_group(); G

In [ ]:
G_gap = gap(G)
G_gap.RelatorsOfFpGroup()

In [ ]:
f = G_gap.GQuotients(PSL(2,7))[1]
C = M.cover(f.Kernel())
C.volume()/M.volume()

In [ ]:
G_mag = magma(G)
H = G_mag.LowIndexSubgroups(12)[174]
N = G_mag.Core(H)
print(G_mag.Index(N))
M.cover(N).homology()

## Verified computation

As pioneered by HIKMOT, SnapPy can use interval arithmetic to *prove* that a given manifold is hyperbolic and provide intervals where the exact shapes must lie.  Uses Sage's complex interval types and the Newton interval method.

In [ ]:
E = snappy.Manifold('K14n1234')
success, shapes = E.verify_hyperbolicity()
success, shapes[:5]

In [ ]:
z0 = shapes[0]
z0.diameter()

In [ ]:
better_shapes = E.verify_hyperbolicity(bits_prec=1000)[1]
max(z.diameter() for z in better_shapes)

By Mostow rigidity, a (finite-volume) hyperbolic structure is unique.  When the manifold has cusps, there is a canonical ideal cell decomposition associated to the hyperbolic structure.  SnapPy uses this to decide when two hyperbolic manifolds are homeomorphic.  

In [ ]:
F = snappy.Manifold('K14n1235')
E.is_isometric_to(F)

That was a numerical calculation and not rigorous because SnapPy might have miscalculated the canonical decompositions.  Let's fix that.

In [ ]:
E.num_tetrahedra(), E.triangulation_isosig()

In [ ]:
E.isometry_signature()

In [ ]:
F.isometry_signature()

**Note:** The latter two strings encode the *homeomorphism type* of these manifolds.

## Link Diagrams: Spherogram

Spherogram is a separately installable module, mostly pure-Python, which deals with knot and link diagrams.  Basic data structure is a planar diagram.

In [ ]:
import spherogram
K = spherogram.random_link(300, 1, consistent_twist_regions=True)
K

In [ ]:
K_orig = K.copy()
K.simplify('global')
K

In [ ]:
K_orig.view(), K.view()

In [ ]:
vol = K.exterior().volume()
vol, vol/3.667

In [ ]:
K.alexander_polynomial()  # 'local' algorithm of Bar-Natan

In [ ]:
L = spherogram.Link('L13n131')

In [ ]:
L.jones_polynomial()

In [ ]:
L.signature()

In [ ]:
L.seifert_matrix()

In [ ]:
D = L.morse_diagram()  # Uses Sage's interface to GLPK

In [ ]:
print(D.is_bridge())
B = D.bridge()
len(B.crossings)

In [ ]:
L.braid_word()

## Spherogram 1.5 and SageMath 7.2 links and braids are friends

In [ ]:
L_sage = L.sage_link()
L_sage

In [ ]:
L_sage_and_back = spherogram.Link(L_sage)

L_sage_and_back.exterior().is_isometric_to(L.exterior())

In [ ]:
w = L.braid_word(as_sage_braid=True)
w, w.parent()

In [ ]:
L_braid = spherogram.Link(braid_closure=w)
L_braid

In [ ]:
L_braid.simplify('global')
L

# Other 3-manifold software.

### Regina

Regina focuses on the purely topological side of things, especially normal surface theory.  Includes some support for higher-dimensional manifolds. Like SnapPy, it has a stand-alone GUI and also a Python interface. 

See http://unhyperbolic.org/sageRegina/ for how to install it painlessly into Sage.

### t3m

Light-weight pure Python library for dealing with triangulations of 3-manifolds.  Included with SnapPy as `snappy.snap.t3mlite`. 

# Possible projects for Sage Days 74

### Make SnapPy (even) easier to install into SageMath

   - Sage optional package?

   - SageMath binaries for OS X < 10.11? 

### Sage's "attach" blocks Tkinter 

In Sage's IPython-based interpreter, the mechanism behind Sage's attach functionality blocks IPython's ability to integrate with Tk (and other GUI's) event loops. (See trac #15152).



### Modernize CyPari

CyPari is based on Sage circa 2012, and there have been improvements since, with more to come (Demeyer et. al.).

### Modularization

Modularize some parts of the SageMath kernel, for example interval arithmetic, for use in stand-alone SnapPy.